In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam

In [46]:
from tensorflow import keras
from tensorflow.keras import Model
from keras.layers import Dense

# 사저학습된 인셉션V3 모델 로드
base_model = InceptionV3(weights='imagenet', include_top=False, pooling='max')

# 드롭아웃
# x = Dropout(0.2)(base_model.output)

# 출력층을 클래스 수에 맞게 수정, 다중 분류로 소프트맥스 함수 사용
output = Dense(4, activation='softmax')(base_model.output)
model = Model(inputs=base_model.input, outputs=output)

In [47]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 전처리
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.3)

# 데이터 불러오기 및 전처리
data_dir = '/content/drive/MyDrive/Colab Notebooks/archive'
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

Found 681 images belonging to 4 classes.
Found 289 images belonging to 4 classes.


In [48]:
# 손실함수, 옵티마이저 세팅
model.compile(optimizer=Adam(learning_rate=0.00005), loss='categorical_crossentropy', metrics=['accuracy'])

# 학습 진행
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // 32,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples // 32,
    epochs = 50)

# 에포크당 정확도 출력
for i in range(len(history.history['accuracy'])):
    print(f"Epoch {i+1}, Train Accuracy: {history.history['accuracy'][i]}, Validation Accuracy: {history.history['val_accuracy'][i]}")

Epoch 1/50
21/21 [==============================] - 48s 893ms/step - loss: 1.7507 - accuracy: 0.4314 - val_loss: 1.8749 - val_accuracy: 0.5556
Epoch 2/50
21/21 [==============================] - 17s 825ms/step - loss: 0.7086 - accuracy: 0.7442 - val_loss: 1.1783 - val_accuracy: 0.5972
Epoch 3/50
21/21 [==============================] - 19s 923ms/step - loss: 0.3599 - accuracy: 0.8795 - val_loss: 1.1671 - val_accuracy: 0.6701
Epoch 4/50
21/21 [==============================] - 18s 840ms/step - loss: 0.2903 - accuracy: 0.8968 - val_loss: 0.9694 - val_accuracy: 0.7222
Epoch 5/50
21/21 [==============================] - 17s 833ms/step - loss: 0.1906 - accuracy: 0.9353 - val_loss: 0.9342 - val_accuracy: 0.7396
Epoch 6/50
21/21 [==============================] - 17s 827ms/step - loss: 0.1376 - accuracy: 0.9569 - val_loss: 0.7719 - val_accuracy: 0.7500
Epoch 7/50
21/21 [==============================] - 23s 1s/step - loss: 0.1133 - accuracy: 0.9676 - val_loss: 0.7830 - val_accuracy: 0.7778
Ep

In [50]:
# 모델 저장
model.save('/content/drive/MyDrive/Colab Notebooks/my_model.h5')

In [56]:
from tensorflow.keras.preprocessing import image
import numpy as np

# 내 이미지 로드
img_path = 'profile(1.1 crop).JPG'  # replace with your image path
img = image.load_img(img_path, target_size=(224, 224))

# 이미지 전처리
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0  # rescale the same as we did for training

# 예측 수행
prediction = model.predict(x)

# 확률 소수점 둘째자리 반올림
rounded_prediction = np.around(prediction, 2)

# 예측 확률 출력
print(rounded_prediction)

1/1 [==============================] - 0s 27ms/step
[[1. 0. 0. 0.]]


In [54]:
# 클래스 인자 획득
class_indices = train_generator.class_indices

# 클래스 이름 가져오기
class_names = list(class_indices.keys())

# 가장 확률 높은 label을 인덱싱
predicted_class_index = np.argmax(prediction)

# 나의 class는?
predicted_class_name = class_names[predicted_class_index]

print(predicted_class_name)

Asian
